## IMPORTS

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]= "0" #imports
#imports
import torch
import torch.nn.functional as F
%config Completer.use_jedi = False
from torch_scatter import scatter

%env CUDA_LAUNCH_BLOCKING 1

env: CUDA_LAUNCH_BLOCKING=1


In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

/tmp/ipykernel_8054/495279268.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
import numpy as np
import torch_geometric
import pysat as ps
# import gurobipy as gp
# from gurobipy import GRB

from pysat.formula import CNF
from matplotlib import pyplot as plt
from pysat.solvers import Lingeling


import math

/home/aspect/anaconda3/envs/extensions/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
from torch.nn import Linear
from IPython.core.debugger import set_trace
from datetime import date
#from kernel.datasets import get_dataset
from itertools import product
import time
from torch import tensor
from torch.optim import Adam
from torch.optim import SGD
from torch.nn import PReLU
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from math import ceil
from torch.nn import Linear
from torch.distributions import categorical
from torch.distributions import Bernoulli
import torch.nn
%matplotlib inline
from matplotlib import pyplot as plt
from torch_geometric.utils import convert as cnv
from torch_geometric.utils import sparse as sp
from torch_geometric.data import Data
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
from torch.utils.data.sampler import RandomSampler
from torch.nn.functional import gumbel_softmax
from torch.distributions import relaxed_categorical
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.inits import glorot, zeros
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
from torch_geometric.nn import GINConv, GATConv, global_mean_pool, NNConv, GCNConv
from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU
from torch_geometric.nn import MessagePassing
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
from torch_geometric.nn import GINConv, global_mean_pool
from torch_geometric.data import Batch 
from torch import autograd
from torch_geometric.utils import to_dense_batch, to_dense_adj
from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops
from torch_geometric.utils import dropout_adj, to_undirected, to_networkx
from torch_geometric.utils import is_undirected
import scipy
import scipy.io
from matplotlib.lines import Line2D
from torch_geometric.utils.convert import from_scipy_sparse_matrix
from networkx.algorithms.approximation import max_clique
import pickle
from torch_geometric.nn import SplineConv, global_mean_pool, DataParallel
from torch_geometric.data import DataListLoader
from random import shuffle
from networkx.algorithms.approximation import max_clique
from networkx.algorithms import find_cliques
from torch_geometric.nn.norm import graph_size_norm
from torch_geometric.utils import dense_to_sparse
from torch_geometric.nn import GatedGraphConv, GINEConv
import os
from pysat.solvers import Lingeling,Minisat22, Glucose4

In [7]:
def old_dense_to_sparse(tensor):
    r"""Converts a dense adjacency matrix to a sparse adjacency matrix defined
    by edge indices and edge attributes.

    Args:
        tensor (Tensor): The dense adjacency matrix.
     :rtype: (:class:`LongTensor`, :class:`Tensor`)
    """
    assert tensor.dim() == 2
    index = tensor.nonzero(as_tuple=False).t().contiguous()
    value = tensor[index[0], index[1]]
    return index, value            
                        
def generate_random_kCNF(k,num_variables, num_clauses):
    formula = CNF()
    for clause_i in range(num_clauses):
        clause = []
        for literal in range(k):
            #randint is not inclusive on the upper bound, hence +1
            literal = np.random.randint(-num_variables,num_variables+1)
            while (literal in clause) or (-literal in clause) or (literal==0):
                    literal = np.random.randint(-num_variables,num_variables+1)
            clause += [literal]
        formula.append(clause)
    return formula

#detect dependencies between clauses (2 clauses have a dependency if they share a variable)
#dependencies are lopsided, a variable and its negative will connect two clauses with a negative sign
def detect_dependency_2(clause_1, clause_2):
    dependency_score = 0
    for  var_1 in clause_1:
        for var_2 in clause_2:
            if np.abs(var_1) == np.abs(var_2):
                if np.sign(var_1)==np.sign(var_2):
                    return 1
                else:
                    return -1
            
    return 0


def detect_dependency(clause_1, clause_2):
    dependency_score = 0
    for  var_1 in clause_1:
        for var_2 in clause_2:
            if np.abs(var_1) == np.abs(var_2):
                if np.sign(var_1)==np.sign(var_2):
                    dependency_score = 1
                else:
                    return -1
            
    return dependency_score

def get_VarClause_matrix(formula,k):
    clause_var_adj = np.zeros((len(formula.clauses)+formula.nv,len(formula.clauses)+formula.nv))
    clause_var_dir = np.zeros((formula.nv,formula.nv+len(formula.clauses)))
    for i in range(len(formula.clauses)):
        for j in range(k):
            literal = formula.clauses[i][j] 
            clause_var_adj[i,np.abs(literal)-1] =  np.sign(literal)
            clause_var_dir[np.abs(literal)-1,i+formula.nv] = np.sign(literal)
    return clause_var_adj, clause_var_dir
    
def get_VarClause_matrix_2(formula,k):
    clause_var_adj = np.zeros((len(formula.clauses)+formula.nv,len(formula.clauses)+formula.nv))
    clause_var_dir = np.zeros((formula.nv,formula.nv+len(formula.clauses)))
    for i in range(len(formula.clauses)):
        for j in range(k):
            literal = formula.clauses[i][j] 
            clause_var_adj[i,np.abs(literal)-1] =  np.sign(literal)
            clause_var_dir[np.abs(literal)-1,i+formula.nv] = np.sign(literal)
    return clause_var_adj, clause_var_dir
    
    
  
def get_dependency_matrix(formula):
    num_clauses = len(formula.clauses)
    dependency_graph_adj = np.zeros((len(formula.clauses)+formula.nv,len(formula.clauses)+formula.nv))
    for row in range(num_clauses):
        for column in range(num_clauses):
            dependency_graph_adj[row+formula.nv,column+formula.nv] = detect_dependency(formula.clauses[row], formula.clauses[column])
    return dependency_graph_adj- np.eye(dependency_graph_adj.shape[0])



def generate_dimacs_CNFDataset(dimacs_dataset, k):
    dataset=[]
    
    for (counter,formula) in enumerate(dimacs_dataset):

        with Glucose4(bootstrap_with=formula.clauses, with_proof=True) as l:
            is_sat = (l.solve())
            
        num_variables = len(set([np.abs(item) for sublist in (formula.clauses) for item in sublist]))
        num_clauses  = len(formula.clauses)

        #adj_matrix = get_dependency_matrix(formula)
        var_clause_mat, var_clause_dirmat = get_VarClause_matrix(formula,k)
        
        #delete isolated node rows,columns from clause-clause graph and var-clause graph
        #adj_matrix = np.delete(adj_matrix,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=0)
        #adj_matrix = np.delete(adj_matrix,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=1)


        var_clause_dirmat_temp = np.delete(var_clause_dirmat,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=0)
        var_clause_dirmat_temp = np.delete(var_clause_dirmat_temp,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=1)

        var_clause_dirmat = var_clause_dirmat_temp
        
                #print(var_clause_dirmat.shape)
        vc_edge_ind, vc_edge_attr = old_dense_to_sparse(torch.tensor(var_clause_dirmat))
        #dirvc_edge_ind, dirvc_edge_attr = dense_to_sparse(torch.tensor(var_clause_dirmat))
        #print("it's undirected: ", is_undirected(vc_edge_ind))
        x_var = torch.zeros(num_variables)
        x_clause = torch.ones(num_clauses)
        x_comb = torch.cat([x_var,x_clause])
        #cc_edge_ind, cc_edge_attr = old_dense_to_sparse(torch.tensor(adj_matrix))


        graph_object = Data(x= x_comb, edge_index = vc_edge_ind, edge_attr = vc_edge_attr, formula = [formula],  sat=is_sat)

#        graph_object = Data(x= x_comb, edge_index = vc_edge_ind, edge_attr = vc_edge_attr, cc_edge_index = cc_edge_ind, cc_edge_att =cc_edge_attr,  formula = [formula],  sat=is_sat)
        dataset += [graph_object]
        print("current graph: ", counter)
    return dataset
        




def generate_RandomCNFDataset(num_graphs, k, num_variables_low, num_clauses_low, num_variables_high=None, num_clauses_high=None, verbose = True):
    dataset=[]
    for i in range(num_graphs):
        if num_variables_high is None:
            num_variables = num_variables_low
        else:
            num_variables = np.random.randint(num_variables_low, num_variables_high)
            
        if num_clauses_high is None:
            num_clauses = num_clauses_low 
        else:
            num_clauses = np.random.randint(num_clauses_low, num_clauses_high)

        num_variables = np.random.randint(num_variables_low, num_variables_high)        
        formula = generate_random_kCNF(k, num_variables,num_clauses)

        
        with Lingeling(bootstrap_with=formula.clauses, with_proof=True) as l:
            is_sat = (l.solve())
            
        num_variables = len(set([np.abs(item) for sublist in (formula.clauses) for item in sublist]))
        num_clauses  = len(formula.clauses)

        #adj_matrix = get_dependency_matrix(formula)
        var_clause_mat, var_clause_dirmat = get_VarClause_matrix(formula,k)
    
        
        # #delete isolated node rows,columns from clause-clause graph and var-clause graph
        # adj_matrix = np.delete(adj_matrix,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=0)
        # adj_matrix = np.delete(adj_matrix,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=1)


        var_clause_dirmat_temp = np.delete(var_clause_dirmat,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=0)
        var_clause_dirmat_temp = np.delete(var_clause_dirmat_temp,np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0],axis=1)

        var_clause_dirmat = var_clause_dirmat_temp


        #print(var_clause_dirmat.shape)
        vc_edge_ind, vc_edge_attr = old_dense_to_sparse(torch.tensor(var_clause_dirmat))
        #dirvc_edge_ind, dirvc_edge_attr = dense_to_sparse(torch.tensor(var_clause_dirmat))
        #print("it's undirected: ", is_undirected(vc_edge_ind))
        x_var = torch.zeros(num_variables)
        x_clause = torch.ones(num_clauses)
        x_comb = torch.cat([x_var,x_clause])
        #cc_edge_ind, cc_edge_attr = old_dense_to_sparse(torch.tensor(adj_matrix))
        
#         if contains_isolated_nodes(vc_edge_ind):
#             print(np.where((np.abs(var_clause_dirmat).sum(1)[:formula.nv]==0))[0])
#             print("contains isolated nodes")
#             breakpoint()
        graph_object = Data(x= x_comb, edge_index = vc_edge_ind, edge_attr = vc_edge_attr, formula = [formula],  sat=is_sat)
#       graph_object = Data(x= x_comb, edge_index = vc_edge_ind, edge_attr = vc_edge_attr, cc_edge_index = cc_edge_ind, cc_edge_att =cc_edge_attr,  formula = [formula],  sat=is_sat)

        dataset += [graph_object]
        if verbose:
            print("current graph: ", i)
    return dataset

In [8]:
def read_dimacs_directory(directory):
    directory_contents = os.listdir(directory)
    dataset_formulas = []
    #print(directory_contents)
    for k in range(len(directory_contents)):
        if "txt" in directory_contents[k].split()[0]:
            formula = CNF(from_file= directory + '/' +directory_contents[k])
            #print(formula)
            dataset_formulas += [formula]  
    return dataset_formulas

## GENERATE OR LOAD DATASET HERE

In [9]:
parent_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
my_data = read_dimacs_directory(parent_path+'/datasets/dimacs_400_random/')
test_dataset_torch = generate_dimacs_CNFDataset(my_data,3)

current graph:  0
current graph:  1
current graph:  2
current graph:  3
current graph:  4
current graph:  5
current graph:  6
current graph:  7
current graph:  8
current graph:  9
current graph:  10
current graph:  11
current graph:  12
current graph:  13
current graph:  14
current graph:  15
current graph:  16
current graph:  17
current graph:  18
current graph:  19
current graph:  20
current graph:  21
current graph:  22
current graph:  23
current graph:  24
current graph:  25
current graph:  26
current graph:  27
current graph:  28
current graph:  29
current graph:  30
current graph:  31
current graph:  32
current graph:  33
current graph:  34
current graph:  35
current graph:  36
current graph:  37
current graph:  38
current graph:  39
current graph:  40
current graph:  41
current graph:  42
current graph:  43
current graph:  44
current graph:  45
current graph:  46
current graph:  47
current graph:  48
current graph:  49
current graph:  50
current graph:  51
current graph:  52
cur

## Decide on model

In [10]:
#from pytorch_memlab import profile, mem_reporter
#@profile
class Erdos_MPNN(torch.nn.Module):
    def __init__(self, num_layers, hidden1, hidden2,  num_iterations = 1):
        super(Erdos_MPNN, self).__init__()
        self.hidden1 = hidden1
        self.hidden2 = hidden2
        self.momentum = 0.1
        self.num_iterations = num_iterations
        self.convs = torch.nn.ModuleList()
        self.numlayers = num_layers
        self.heads = 1
        self.concat = True        
        self.bns = torch.nn.ModuleList()
        self.bn = BN(self.heads*self.hidden1, momentum=self.momentum)
        for i in range(num_layers-1):
            self.bns.append(BN(self.heads*self.hidden1, momentum=self.momentum))


        self.convs = torch.nn.ModuleList()        
        self.bn1 = BN(self.heads*self.hidden1)
       
        self.conv  = GatedGraphConv(self.hidden1,self.numlayers)
        

        self.conv1 = GINEConv(Sequential(Linear(self.hidden2,  self.heads*self.hidden1),
            ReLU(),
            Linear( self.heads*self.hidden1,  self.heads*self.hidden1),
            ReLU(),
            BN(self.heads*self.hidden1, momentum=self.momentum),
        ),train_eps=False)
        
        self.x_lin_2 = Linear(self.hidden1,1)
        self.x_lin_1 = Linear(self.hidden1,self.hidden1)
        
        self.x_batchnorm = BN(self.hidden1)
        if self.concat:
            self.lin1 = Linear(self.heads*self.hidden1, self.hidden1)
        else:
            self.lin1 = Linear(self.hidden1, self.hidden1)
        self.lin2 = Linear(self.hidden1, 1)
        self.gnorm = graph_size_norm.GraphSizeNorm()

                    


    def reset_parameters(self):
        self.conv1.reset_parameters()
        #self.conv_alt.reset_parameters()
        #self.conv_alt_rec.reset_parameters()
        self.conv.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
            
        for bn in self.bns:
            bn.reset_parameters()
            
        self.bn1.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()
       # self.x_lin_1.reset_parameters()
        #self.x_lin_2.reset_parameters()
#
        # self.x_conv_probs.reset_parameters()
        # self.x_conv_degs.reset_parameters()

        
        #self.x_conv_2.reset_parameters()
        self.x_batchnorm.reset_parameters()


    def forward(self, data):
        x_index = data.x
        batch = data.batch
        num_graphs = batch.max().item() + 1

#         print("degs shape", degs.shape)
#         print("data.x shape ", data.x.shape)
        
        #directed edge index
        #print(is_undirected(data.edge_index))
        d_row, d_col = data.edge_index.detach()
        edge_attr = data.edge_attr.detach()
        
        pos_assoc = edge_attr*((edge_attr>0)*1.)
        neg_assoc = edge_attr*((edge_attr<0)*1.)
        
        #make undirected
        edge_index = torch.cat([data.edge_index,data.edge_index[[1,0],:]],dim=1)
        edge_attr = torch.cat([edge_attr.unsqueeze(-1), edge_attr.unsqueeze(-1)],dim=0)

        
        select_clauses = (data.x==1)
        select_variables = (data.x==0)

        #CREATE UNDIRECTED EDGE_INDEX
        row, col = edge_index
        
        #DANGEROUS
        degs = degree(row)
        x = degs*1.
        

        #other stuff
        total_num_edges = edge_index.shape[1]
        N_size = x.shape[0]
        
        # #positive and negative degrees of each node
        posdeg= scatter(pos_assoc,d_col, dim=0, dim_size=data.x.shape[0], reduce="sum")+scatter(pos_assoc,d_row, dim=0, dim_size=data.x.shape[0], reduce="sum")
        negdeg= scatter(neg_assoc,d_col, dim=0, dim_size=data.x.shape[0], reduce="sum")+scatter(neg_assoc,d_row, dim=0, dim_size=data.x.shape[0], reduce="sum")
        

        x = torch.cat([posdeg.unsqueeze(-1), negdeg.unsqueeze(-1)],dim=1)*1.
        x = x.float()

        ##UNSQUEEZED
        #x = F.leaky_relu(self.conv1(x.unsqueeze(-1), edge_index))# +x
        adjusted_ea = torch.repeat_interleave(edge_attr,self.hidden2,1)
        adjusted_ea = adjusted_ea.float()
        x = x.squeeze(-1)
        x = x.float()
        
        iterations = 0
        total_loss = []
        while (iterations < self.num_iterations):
            
            if(iterations>0):
                x = torch.cat([posdeg.unsqueeze(-1), negdeg.unsqueeze(-1)],dim=1)*1.
                x = x.float()
        
                recurr =  torch.repeat_interleave(transformed_probs_comp.unsqueeze(-1),self.hidden2,1)
                recurr_activ =  F.leaky_relu(self.conv_alt_rec(recurr, edge_index,adjusted_ea))
                x = F.leaky_relu(self.conv_alt(x, edge_index,adjusted_ea))  * recurr_activ
                                 
            else:
                #FIRST CONVO
                x = F.leaky_relu(self.conv1(x, edge_index,adjusted_ea))# +x

            x_1 = x.clone()
            
            #ADJUST EDGE 
            adjusted_ea_2 = torch.repeat_interleave(edge_attr.unsqueeze(-1), self.hidden1,1)
            adjusted_ea_2 = (adjusted_ea_2.squeeze(-1)).float()


            x =  x+F.leaky_relu(self.conv(x, edge_index,edge_attr.float())) #+ x_1 GRU MODE

            x =  F.leaky_relu(self.lin1(x)) 
            x = F.leaky_relu(self.lin2(x)) 

            probs = torch.sigmoid(x)*0.999990 + 0.000001 
            probs = probs.squeeze(-1) #shape: x
            probs_backup = probs.clone()

            #sanity check
            #WARNING EDGE ATTR!= DATA.EDGE_ATTR
            pos_neg_index = (data.edge_attr==-1)*1.
            test_probs = torch.ones_like(probs)*0.25
            test_probs = test_probs.squeeze(-1)
            test_msg = (1-pos_neg_index) *(1- test_probs[d_row]) + (pos_neg_index)*(test_probs[d_row])
            test_msg_c = (pos_neg_index) *(1- test_probs[d_row]) + (1-pos_neg_index)*(test_probs[d_row])

            msg = pos_neg_index *(1- probs[d_row]) + (1-pos_neg_index)*(probs[d_row])
            comp_msg = pos_neg_index *( probs[d_row]) + (1-pos_neg_index)*(1-probs[d_row])
            pre_exp_comp = scatter(torch.log(comp_msg),d_col,dim=0, reduce="sum")
            pre_exp = scatter(torch.log(msg),d_col,dim=0, reduce="sum")
            transformed_probs = torch.exp(pre_exp)
            transformed_probs_comp = torch.exp(pre_exp_comp)
            clause_probs = transformed_probs.squeeze(-1)*((x_index==1)*1.)
            clause_probs_comp = transformed_probs_comp.squeeze(-1)*((x_index==1)*1.) #shape: x
            var_probs = probs[data.x==0].detach()

            loss_comp = scatter(clause_probs, batch, dim=0 , reduce="sum") #expected_loss#*normalize
            loss = scatter(clause_probs_comp, batch, dim=0 , reduce="sum") #expected_loss#*normalize
            
            num_clauses = (map(len, [i[0].clauses for i in data.formula]))
            uniform_prob = torch.tensor([0.125*i for i in num_clauses]) #union bound for uniform probability

            total_loss += [loss.mean()]
            
            iterations += 1

            
        exponents = torch.arange(self.num_iterations,device='cuda')
        
                ## NUM CLAUSES, C/V RATIO, LLL SAT RATIO 
        num_vars = torch.tensor([(data.x[data.batch==num]==0).sum().item() for num in data.batch.unique()], device='cuda')
        num_clauses =  torch.tensor([(data.x[data.batch==num]==1).sum().item() for num in data.batch.unique()], device='cuda')
        LLL_check =  clause_probs_comp
        LLL_diff = clause_probs_comp[select_clauses]
        LLL_loss = scatter(LLL_diff, batch[select_clauses], dim=0 , reduce="sum") #shape: 32
        UnionBoundCheck = (LLL_loss<=1.)*1.
        
        #breakpoint()
        #### RATIOS/STATS
        c_v_ratio = num_clauses/num_vars
        clauses_satisfied = scatter(LLL_check*1., batch, dim=0 , reduce="sum")        
        perc_of_clauses_satisfied_per_graph = clauses_satisfied/num_clauses


        retdict = {}
        ub_term = LLL_loss
        final_loss = ub_term # + LLL_gamma*LLL_prob
        
        #################################
        #scatter plot
        hardness_unionbound_scatterplot = torch.cat([c_v_ratio.unsqueeze(-1), loss.unsqueeze(-1)],dim=1)


        #color
        color = (c_v_ratio.unsqueeze(-1)).detach()
        color = color/color.max().item()
        color = torch.round(color*255.)
        dummy = torch.zeros_like(color)
        color_r = torch.cat([color ,dummy,dummy],dim=1)
        color_g = torch.cat([dummy,color, dummy],dim=1)
        

        is_sat = torch.tensor([sat*1. for sat in data.sat],device='cuda').unsqueeze(-1)
        final_color = is_sat * color_g  + (1-is_sat)*color_r

        retdict["output"] = [probs.squeeze(-1),"hist"]   #output
        retdict["union bound avg"] = [LLL_loss.mean().squeeze(),"sequence"] #final loss
        retdict["loss"] = [final_loss.mean().squeeze(),"sequence"] #final loss
        retdict["union bound"] = [loss.mean().squeeze(),"sequence"] #final loss
        retdict["loss_comp"] = [loss_comp.mean().squeeze(),"null"] #final loss
        # retdict["percent LLL constraints satisfied"] =  [true_perc_of_clauses_satisfied_per_graph,'hist']
        # retdict["average percentage of LLL constraints satisfied"] =  [true_perc_of_clauses_satisfied_per_graph.mean(),'sequence']
        retdict["union bound/hardness (c/v ratio)"] = [hardness_unionbound_scatterplot, 'scatter',final_color]
        #retdict["LLL bound sat percentage/hardness (c/v ratio)"] = [hardness_lllbound_scatterplot, 'scatter',final_color]
        retdict["clause_probs"] = [clause_probs_comp[data.x==1], "hist"]
        retdict["clause_probs_full"] = [clause_probs_comp, "null"]
        retdict['variable probs'] = [var_probs.squeeze(),'hist']
        retdict["variable_probs_full"] = [probs,'null']
        retdict["union bounds"] = [LLL_loss.squeeze(),'hist']
        retdict["union bound sat"] = [UnionBoundCheck.squeeze(),'hist']

        
        return retdict
    
    def __repr__(self):
        return self.__class__.__name__


## TRAINING CODE HERE

In [11]:
numlayers=2
receptive_field= numlayers + 1
val_losses = []
cliq_dists = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr_decay_step_size = 20
lr_decay_factor = 0.95
lr_lower_bound = 0.00001


## Dataset preparation

In [12]:
!mkdir 'ErdoSAT_repo'
#for validation checkpoints

In [13]:
##validation set
big_large_easy_validation = generate_RandomCNFDataset(300,3,100,400,num_variables_high=101, num_clauses_high=401);
infinite_data = True
if not infinite_data:
    big_large_easy_train = generate_RandomCNFDataset(1500,3,100,400,num_variables_high=101, num_clauses_high=401);

current graph:  0
current graph:  1
current graph:  2
current graph:  3
current graph:  4
current graph:  5
current graph:  6
current graph:  7
current graph:  8
current graph:  9
current graph:  10
current graph:  11
current graph:  12
current graph:  13
current graph:  14
current graph:  15
current graph:  16
current graph:  17
current graph:  18
current graph:  19
current graph:  20
current graph:  21
current graph:  22
current graph:  23
current graph:  24
current graph:  25
current graph:  26
current graph:  27
current graph:  28
current graph:  29
current graph:  30
current graph:  31
current graph:  32
current graph:  33
current graph:  34
current graph:  35
current graph:  36
current graph:  37
current graph:  38
current graph:  39
current graph:  40
current graph:  41
current graph:  42
current graph:  43
current graph:  44
current graph:  45
current graph:  46
current graph:  47
current graph:  48
current graph:  49
current graph:  50
current graph:  51
current graph:  52
cur

In [14]:
#####################TODOS:
b_sizes = [16]
l_rates = [0.001]
depths = [10]
#rand_seeds = list(range(70,80))
rand_seeds = [3]
widths = [128]

num_iterations_per_epoch = 10

wandb_logging = False  
rnn_its = 1
epochs = 1000
retdict = {}
edge_drop_p = 0.0
edge_dropout_decay = 0.90
validation_timeout = 75
vis_batch = None
vis_batches = []
performance_monitor = []
curr_date = date.today()
curr_date = curr_date.strftime("%B %d, %Y")
curr_LR = None
for batch_size, learning_rate, numlayers,  r_seed, hidden_1 in product(b_sizes, l_rates, depths, rand_seeds, widths):

    torch.manual_seed(r_seed)
    valdata = big_large_easy_validation
    val_loader =  DataLoader(valdata, batch_size, shuffle=False)

#####Set up data and loaders 
    if not infinite_data:
        traindata= big_large_easy_train
        train_loader = DataLoader(traindata, batch_size, shuffle=True)

#####Set up model and relevant vars
    #num_sat_instances =  sum([1*data.sat for data in traindata])
    val_losses = []
    cliq_dists = []
    #hidden_1 = 128
    hidden_2 = 2
    net =  Erdos_MPNN(num_layers = numlayers, hidden1= hidden_1, hidden2 = hidden_2 ,num_iterations=rnn_its)
    net.to(device).reset_parameters()
    optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.00000)
    net.train()
    graphs_solved_LLL = 0
    graphs_solved_LLL_previous = 0
    #with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        count = 0
        avg_epoch_loss = 0
        totalretdict = {}
        #learning rate schedule
        if epoch % lr_decay_step_size == 0:
            for param_group in optimizer.param_groups:
                        new_lr = lr_decay_factor * param_group['lr']
                        if new_lr > lr_lower_bound:
                            param_group['lr'] = new_lr
                        else:
                            param_group['lr'] = lr_lower_bound
                        if (param_group['lr'] <= lr_lower_bound) and (performance_monitor[-1] <= 0.999*performance_monitor[-50]):
                            param_group['lr'] = param_group['lr']*1

                        curr_LR =  param_group['lr']
                
            val_loss = 0
            with torch.no_grad():
                net.eval()
                for data in val_loader:
                    count += 1 
                    optimizer.zero_grad(), 
                    data = data.to(device)
                    retdict = net(data)
                    val_loss += retdict["loss"][0]/len(val_loader)
                    optimizer.zero_grad()
        
            print("Current Validation Loss: ", val_loss)
            val_losses += [val_loss]
            #parent_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
            PATH = "ErdoSAT_repo/"+str(val_loss)+str(epoch)+"ErdoSATmodel.pt"
            if ((val_loss<=torch.stack(val_losses))*1.).prod():
                 print("SAVED!")
                 torch.save({'epoch': epoch,'model_state_dict': net.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'loss': val_loss}, PATH)                 



        ####Infinite data
        if infinite_data:
            traindata = generate_RandomCNFDataset(num_iterations_per_epoch*batch_size,3,100,400,num_variables_high=101, num_clauses_high=401,verbose=False);
            train_loader = DataLoader(traindata, batch_size, shuffle=True)


        print('Epoch: ', epoch)
        net.train()
        for data in train_loader:
            count += 1 
            optimizer.zero_grad(), 
            data = data.to(device)
            retdict = net(data)
            loss = retdict["loss"][0]
            avg_epoch_loss += loss/len(train_loader)
            for key,val in retdict.items():
                if "sequence" in val[1]:
                    if key in totalretdict:
                        totalretdict[key][0] += val[0].item()
                    else:
                        totalretdict[key] = [val[0].item(),val[1]]
            if epoch > 1:
                    loss.backward()
                    #print(retdict["loss"][0])
                    #torch.nn.utils.clip_grad_norm_(net.parameters(),4.)
                    optimizer.step()
            else:
                    optimizer.zero_grad()
        print("Average Epoch Loss (avg unsat clauses): ", avg_epoch_loss)

        


/home/aspect/anaconda3/envs/extensions/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Current Validation Loss:  tensor(48.7261, device='cuda:0')
SAVED!
Epoch:  0
Average Epoch Loss:  tensor(51.6350, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  1
Average Epoch Loss:  tensor(51.7864, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  2
Average Epoch Loss:  tensor(41.4148, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  3
Average Epoch Loss:  tensor(34.8449, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  4
Average Epoch Loss:  tensor(32.1847, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  5
Average Epoch Loss:  tensor(31.1938, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  6
Average Epoch Loss:  tensor(30.2098, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  7
Average Epoch Loss:  tensor(29.7994, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  8
Average Epoch Loss:  tensor(29.0614, device='cuda:0', grad_fn=<AddBackward0>)
Epoch:  9


KeyboardInterrupt: 

In [27]:
###LOAD CHECKPOINT (can skip that step)
checkpoint = torch.load('ErdoSAT_repo/tensor(6.2546980ErdoSATmodel.pt')
net.load_state_dict(checkpoint['model_state_dict'])
net.eval()

Lovasz_MPNN

In [28]:
dataset = test_dataset_torch
test_loader = DataLoader(test_dataset_torch, batch_size, shuffle=False)
expected_unsat_clauses_before =  []
unsat_clauses_after = []

batch_counter = 0

for data in test_loader:
    net.eval()
    data = data.to(device)
    retdict = net(data)
    print("Current batch: ", batch_counter)

    batch_probs = retdict['output'][0]
    new_batch_probs = batch_probs.detach()

    for graph_no,graph in enumerate(data.to_data_list()):
        probs = batch_probs[data.batch==graph_no]
        select_vars = (graph.x==0)
        select_clauses = (graph.x==1)
        d_row, d_col = graph.edge_index.detach()
        edge_attr = graph.edge_attr.detach()
        pos_assoc = edge_attr*((edge_attr>0)*1.)
        neg_assoc = edge_attr*((edge_attr<0)*1.)
        pos_neg_index = (graph.edge_attr==-1)*1.
        new_probs = probs.detach()
        new_var_probs = new_probs[select_vars]
        


        comp_msg = pos_neg_index *( new_probs[d_row]) + (1-pos_neg_index)*(1-new_probs[d_row])
        pre_exp_comp = scatter(torch.log(comp_msg),d_col,dim=0, reduce="sum")
        transformed_probs_comp = torch.exp(pre_exp_comp)
        clause_probs_comp = transformed_probs_comp.squeeze(-1)*((select_clauses)*1.) #shape: x
        clause_probs_comp  = clause_probs_comp[select_clauses]
        running_expected_clause_violation = clause_probs_comp.sum()
        expected_unsat_clauses_before +=  [running_expected_clause_violation]


        for counter in range(len(new_var_probs)):
            if (new_var_probs[counter]<=1e-4):
                new_var_probs[counter]=0.
                continue
            elif (new_var_probs[counter]>0.9999):
                new_var_probs[counter]=1.
                continue
            p1 = new_var_probs[counter].detach()
            one_minus_p1 = 1-p1
            new_var_probs[counter] = 1.
            #print("new probs value2", new_var_probs[counter])

            comp_msg = pos_neg_index *(new_var_probs[d_row]) + (1-pos_neg_index)*(1-new_var_probs[d_row])
            pre_exp_comp = scatter(torch.log(comp_msg),d_col,dim=0, reduce="sum")
            transformed_probs_comp = torch.exp(pre_exp_comp)
            clause_probs_comp = transformed_probs_comp.squeeze(-1)*((select_clauses)*1.) #shape: x
            clause_probs_comp  = clause_probs_comp[select_clauses]
            expected_clause_violation_1 = clause_probs_comp.sum()

            
            if expected_clause_violation_1<=running_expected_clause_violation:
                #print("I'm here1")
                new_var_probs[counter] = 1.
                running_expected_clause_violation = expected_clause_violation_1
            else:
                #print("ACTUALLY")
                new_var_probs[counter] = 0.
                comp_msg = pos_neg_index *(new_var_probs[d_row]) + (1-pos_neg_index)*(1-new_var_probs[d_row])
                pre_exp_comp = scatter(torch.log(comp_msg),d_col,dim=0, reduce="sum")
                transformed_probs_comp = torch.exp(pre_exp_comp)
                clause_probs_comp = transformed_probs_comp.squeeze(-1)*((select_clauses)*1.) #shape: x
                clause_probs_comp  = clause_probs_comp[select_clauses]
                expected_clause_violation_0 = clause_probs_comp.sum()
               # expected_clause_violation_0 = (running_expected_clause_violation - p1*expected_clause_violation_1)/one_minus_p1
                running_expected_clause_violation = expected_clause_violation_0
            #print("new probs value3", new_var_probs[counter])


        unsat_clauses_after +=[running_expected_clause_violation]
            #var_probs = probs[select_vars].detach()
        new_batch_probs[data.batch==graph_no] = new_probs

    batch_counter +=1
    #calculate for 1

        

/home/aspect/anaconda3/envs/extensions/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Current batch:  0
Current batch:  1
Current batch:  2
Current batch:  3
Current batch:  4
Current batch:  5
Current batch:  6
Current batch:  7
Current batch:  8
Current batch:  9
Current batch:  10
Current batch:  11
Current batch:  12
Current batch:  13
Current batch:  14
Current batch:  15
Current batch:  16
Current batch:  17
Current batch:  18
Current batch:  19
Current batch:  20
Current batch:  21
Current batch:  22
Current batch:  23
Current batch:  24
Current batch:  25
Current batch:  26
Current batch:  27
Current batch:  28
Current batch:  29
Current batch:  30
Current batch:  31
Current batch:  32
Current batch:  33
Current batch:  34
Current batch:  35
Current batch:  36
Current batch:  37
Current batch:  38
Current batch:  39
Current batch:  40
Current batch:  41
Current batch:  42
Current batch:  43
Current batch:  44
Current batch:  45
Current batch:  46
Current batch:  47
Current batch:  48
Current batch:  49
Current batch:  50
Current batch:  51
Current batch:  52
Cur

In [34]:
##SANITY CHECK: print 1 if scores after decoding must be equal or better than before decoding (expected behavior)
((torch.stack(expected_unsat_clauses_before)>=torch.stack(unsat_clauses_after))*1.).prod()


tensor(1., device='cuda:0')

In [30]:
print(f'Number of unsat clauses is {torch.stack(unsat_clauses_after).mean()}+/-{torch.stack(unsat_clauses_after).std()}')

Number of unsat clauses is 6.079092502593994+/-2.0700221061706543
